#  Import Required Modules

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.nn import GCNConv
from sklearn.metrics import accuracy_score
from torch_geometric.datasets import Planetoid
from torch_geometric.transforms import NormalizeFeatures

# Fetch Data

In [2]:
cora = Planetoid(root='/tmp/Cora', name='Cora', transform=NormalizeFeatures())

# Define Model

In [3]:
class GCNModel(nn.Module):
    def __init__(self, num_features, hidden_size, num_classes, dropout_rate):
        super(GCNModel, self).__init__()
        self.conv1 = GCNConv(num_features, hidden_size)
        self.conv2 = GCNConv(hidden_size, num_classes)
        self.dropout = nn.Dropout(p=dropout_rate)

        nn.init.xavier_uniform_(self.conv1.lin.weight.data.T)
        nn.init.xavier_uniform_(self.conv2.lin.weight.data.T)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = torch.relu(x)
        x = self.dropout(x)
        x = self.conv2(x, edge_index)
        x = self.dropout(x)
        x = torch.softmax(x, dim=1)
        return x

## Train and Evaluate The Model

In [4]:
def evaluate_model(model, data, test_mask):
    model.eval()
    with torch.no_grad():
        logits = model(data.x, data.edge_index)
        predicted_labels = logits.argmax(dim=1)

    accuracy = accuracy_score(data.y[test_mask].numpy(), predicted_labels[test_mask].numpy())
    return accuracy

In [5]:
def train_model(model, data, train_mask, val_mask, test_mask, patience=10, max_epochs=200):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.01)

    best_val_loss = float('inf')
    best_model = None
    consecutive_no_improvement = 0

    for epoch in range(max_epochs):
        model.train()
        optimizer.zero_grad()
        out = model(data.x, data.edge_index)
        loss = criterion(out[train_mask], data.y[train_mask])
        loss.backward()
        optimizer.step()

        model.eval()
        with torch.no_grad():
            val_loss = criterion(out[val_mask], data.y[val_mask])

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model = model
            consecutive_no_improvement = 0
        else:
            consecutive_no_improvement += 1

        if consecutive_no_improvement == patience:
            print(f'Early stopping at epoch {epoch + 1} due to no improvement in validation loss.')
            break
        
    test_accuracy = evaluate_model(best_model, data, test_mask)
    print(f'Test Accuracy: {test_accuracy:.4f}')

    return best_model

In [6]:
cora_data = cora[0]
num_nodes = cora_data.num_nodes
train_mask = cora_data.train_mask
val_mask = cora_data.val_mask
test_mask = cora_data.test_mask

In [7]:
gcn_model = GCNModel(num_features=cora.num_features,
                     hidden_size=32,
                     num_classes=cora.num_classes,
                     dropout_rate=0.2)

trained_gcn_model = train_model(gcn_model, cora_data, train_mask, val_mask, test_mask)

Early stopping at epoch 83 due to no improvement in validation loss.
Test Accuracy: 0.7950
